# PROBLEM STATEMENT


1) Scrape 20,000 rows of data

2) Scrape the reviews and ratings of laptops, Phones, Headphones, smart watches, Professional Cameras, Printers, Monitors, Home-theater, Router.

3) Scrape the above data from ecommerce sites.

4) Scrape for reviews and ratings.

END RESULT: BUILD AN APP WHICH CAN PREDICT THE RATING BY SEEING THE REVIEW


In [3]:
# importing the libraries that we may need

import pandas as pd
import  requests
import time
import selenium

from  bs4 import BeautifulSoup
from selenium import webdriver

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

1) Scraping Flipkart for reviews and ratings

In [4]:
driver = webdriver.Chrome(r"C:\Users\91950\Downloads\chromedriver_win32 (1)\chromedriver.exe")
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(2)

#when the website is opened, it gives a pop-up which we need to close

driver.find_element_by_xpath("//div[@class='_2QfC02']/button").click()

srch_items = ['laptops', 'Phones','smart watches','Monitors']
title = []
review_text = []
ratings = []

In [5]:
def scrap():    
        for i in driver.find_elements_by_xpath("//div[@class='t-ZTKy']"):
            review_text.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']"):
            title.append(i.text)
        for i in driver.find_elements_by_xpath("//p[@class='_2-N8zT']/../div"):
            ratings.append(i.text)
        return

In [6]:
urls=[]
for i in srch_items:
    # Find the search bar
    srchBar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
    srchBar.clear()
    srchBar.send_keys(i)
    # Clicking on the search button
    driver.find_element_by_xpath("//button[@class='L0Z3Pu']").click()
    time.sleep(3)
    page = []
    for i in driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a"):
        page.append(i.get_attribute('href'))
    for i in page[0:4]:
        driver.get(i)
        time.sleep(3)
        items = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
        for i in items:
            urls.append(i.get_attribute('href'))
len(urls)          

320

In [7]:
for i in urls:
    driver.get(i)
    time.sleep(2)
    for _ in range(2):
        driver.execute_script("window.scrollBy(0,6000)")
        time.sleep(1)
    # Clicking on all reviews
    try:
        btn=driver.find_element_by_xpath("//div[@class='_2c2kV-']/following::a")
        lnk = btn.get_attribute('href')
        driver.get(lnk)
        time.sleep(1)
    except NoSuchElementException:
        pass

    scrap()        
    try:
        n_page=driver.find_elements_by_xpath("//nav[@class='yFHi8N']/a")
        np=[]
        for i in n_page:
            np.append(i.get_attribute('href'))
        for i in np[0:18]:
            driver.get(i)
            time.sleep(1)
            scrap()
    except: continue
len(ratings), len(review_text), len(title) 

(13308, 13308, 13308)

In [8]:
# Creating a dataframe
data = list(zip(title,review_text,ratings))
df = pd.DataFrame(data, columns = ["Review_title","Review_text","Ratings"])
df

,Review_title,Review_text,Ratings
0,Fair,"After buying, everything is good. intel i3 112...",3
1,Pretty good,Smooth performance,4
2,Good,It's Nice This is first day .,3
3,Good choice,This one js nice on time delivery,4
4,Power at a reasonable price,Looks promising. i3 11th Gen (1125G4) with 4 c...,5
...,...,...,...
13303,Highly recommended,Best product & awosome picture quality,5
13304,Good quality product,"Nice product. good packing. nice pictire, Unfo...",4
13305,Must buy!,Value for money....\nDecent viewing angles\nId...,5
13306,Just wow!,Super quality,5


In [10]:
# Saving the data into csv file
df.to_csv(r"C:\Users\91950\Documents\Ratings Prediction\reviews.csv")